In [2]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
import pandas as pd
import glob

In [3]:
autoencoder = tf.keras.saving.load_model('autoencoder_model')
checkpoint_path = 'autoencoder.ckpt'
autoencoder.load_weights(checkpoint_path)
autoencoder.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 128, 176, 1)]     0         
                                                                 
 conv2d_39 (Conv2D)          (None, 128, 176, 256)     2560      
                                                                 
 max_pooling2d_21 (MaxPooli  (None, 64, 88, 256)       0         
 ng2D)                                                           
                                                                 
 conv2d_40 (Conv2D)          (None, 64, 88, 64)        147520    
                                                                 
 max_pooling2d_22 (MaxPooli  (None, 32, 44, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_41 (Conv2D)          (None, 16, 22, 32)        1846

In [5]:
# Create a new model that only includes the encoder part
encoder_model = tf.keras.Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('dense_14').output)

In [6]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt

def audio_to_mel_spectrogram(audio_file_path, max_length=4*22500, sr=22500, n_mels=256):
    #audio_file_path = audio_file_path if type(audio_file_path) == str else audio_file_path.decode('ASCII')
    #audio_file_path = str(audio_file_path).replace('\\\\', os.sep)
    #audio_file_path = audio_file_path.replace('F:', '/mnt/f').replace('//', '/')
    
    with open(audio_file_path, 'rb') as fh:
        y, sr = librosa.load(fh, sr=sr)
    
    if len(y) > max_length:
        # If the audio is longer than the maximum length, cut it
        y = y[:max_length]
    elif len(y) < max_length:
        # If the audio is shorter, pad it with zeros
        pad_length = max_length - len(y)
        y = np.pad(y, (0, pad_length), mode='constant')
    
    # Calculate the Mel spectrogram
    mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_mels)
    mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
    
    return mel_spectrogram

# Generator function for lazy loading of audio data
def audio_data_generator(audio_files):
    for audio_file in audio_files:
        sample = (audio_to_mel_spectrogram(audio_file) + 80)/80
        yield sample.reshape((1, 128, 176, 1))

In [7]:
cls_train = pd.read_csv('cls_train_ids.csv')
cls_val = pd.read_csv('cls_val_ids.csv')

In [13]:
cls_train_sample = cls_train.sample(n=10000)
cls_train_sample.to_csv('cls_train_ids_sample.csv', index=False)

cls_val_sample = cls_val.sample(n=5000)
cls_val_sample.to_csv('cls_val_ids_sample.csv', index=False)

In [14]:
%%time
cls_train_generator = tf.data.Dataset.from_generator(audio_data_generator, args=[cls_train_sample['full_path']], output_signature=tf.TensorSpec(shape=(1, 128, 176, 1)))
np.savetxt('cls_train_x.csv', encoder_model.predict(cls_train_generator), delimiter=",")

      3/Unknown - 0s 50ms/step 

c:\Users\gabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\librosa\feature\spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


10000/10000 [==============================] - 749s 75ms/step
CPU times: total: 15min 18s
Wall time: 12min 42s


In [15]:
%%time
cls_val_generator = tf.data.Dataset.from_generator(audio_data_generator, args=[cls_val_sample['full_path']], output_signature=tf.TensorSpec(shape=(1, 128, 176, 1)))
np.savetxt('cls_val_x.csv', encoder_model.predict(cls_val_generator), delimiter=",")

5000/5000 [==============================] - 399s 80ms/step
CPU times: total: 7min 40s
Wall time: 6min 45s
